In [7]:
import os
import time
import re
from typing import List
from datetime import datetime
import numpy as np

import gymnasium as gym
from gymnasium.wrappers import AtariPreprocessing, FrameStack, RecordEpisodeStatistics, RecordVideo

from stable_baselines3.common.buffers import ReplayBuffer
from stable_baselines3.common.save_util import load_from_pkl, save_to_pkl

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter
from typing import Dict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def step_trigger(step: int):
    return step % 400_000 == 0

def make_env(env_name="ALE/Pong-v5", seed=42):
    env = gym.make(env_name, render_mode="rgb_array", full_action_space=False, frameskip=1)
    env = AtariPreprocessing(env)
    env = FrameStack(env, 4)
    env = RecordEpisodeStatistics(env)
    # A video will be recorded every 400,000 steps.
    # (Can't use lambda expression because it is not supported by cloud pickle)
    env = RecordVideo(env, "runs/videos/", step_trigger=step_trigger, video_length=1000)
    env.observation_space.seed(seed)
    env.action_space.seed(seed)

    return env

In [9]:
class DQN(nn.Module):
    def __init__(self, num_channels, num_actions):
        super(DQN, self).__init__()

        self.conv1 = nn.Conv2d(num_channels, 32, stride=4, kernel_size=8)
        self.conv2 = nn.Conv2d(32, 64, stride=2, kernel_size=4)
        self.conv3 = nn.Conv2d(64, 64, stride=1, kernel_size=3)
        self.flatten = nn.Flatten()

        # 64 * 7 * 7 is the output of last conv layer because of the formula above, for an input of 84*84
        self.linear = nn.Linear(64 * 7 * 7, 512)
        self.head = nn.Linear(512, num_actions)  # Head layer

    def forward(self, x):
        x = x.float() / 255  # Rescale input from [0, 255] to [0, 1]
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.flatten(x)
        x = F.relu(self.linear(x))
        return self.head(x)

- **policy network**: $Q^{A}_{\theta}$
- **target network**: $Q^{B}_{\theta}$

In [10]:

def linear_schedule(start_epsilon: float, end_epsilon: float, duration: int, timestep: int):
    slope = (end_epsilon - start_epsilon) / duration
    return max(slope * timestep + start_epsilon, end_epsilon)

In [11]:
def get_latest_checkpoint_file(files: List[str]) -> any:
    """
    Return the most recent checkpoint file from the passed list of files.

    If multiple files with same datetime are passed, only the first is returned

    :param files: list of file names containing a formatted datetime (=> %d-%m-%Y_%H:%M:%S)
    :return: the file with the most recent date time or ``None`` if no files were found (because of the lack of correctly formatted date in the file name)
    """
    datetime_regex = r"\d{2}-\d{2}-\d{4}_\d{2}:\d{2}:\d{2}"

    latest_file = None
    latest_datetime = datetime.min
    for file in files:
        match = re.search(datetime_regex, file)
        if not match: continue # Go to next element in list if no match is found

        file_datetime = datetime.strptime(match.group(), "%d-%m-%Y_%H:%M:%S")
        if file_datetime > latest_datetime:
            latest_datetime = file_datetime
            latest_file = file

    return latest_file


class DQNAgent:
    def __init__(self, name="CNN_DDQN_Pong-v5", env_name="ALE/Pong-v5"):
        self.name = name

        self.env = make_env(env_name)

        self.start_datetime = None
        self.start_time = None

        # I use the same hyperparameters as this model: https://huggingface.co/sb3/dqn-PongNoFrameskip-v4
        self.MAX_TIMESTEPS = 10_000_000  # Maximum number of total steps
        self.TARGET_UPDATE_INTERVAL = 1000  # Number of steps between the synchronisation of q and target network
        self.LEARNING_STARTS = 100_000  # The number of steps to wait before we start the training, so the agent can explore and store its experience in the replay buffer

        self.TRAIN_FREQUENCY = 4 # Training is done each 4 steps

        self.CHECKPOINT_INTERVAL_EPISODE = 1000 # Checkpoint saving interval per episode (a checkpoint will be saved each X episodes)

        self.REPLAY_SIZE = 10_000
        self.BATCH_SIZE = 32

        self.GAMMA = 0.99  # Discount rate

        self.EXPLORATION_FRACTION = 0.1  # The fraction of 'TOTAL_TIMESTEPS' it takes from 'EPSILON_START' to 'EPSILON_END'.
        self.EPSILON_INITIAL = 1.0
        self.EPSILON_FINAL = 0.01

        self.epsilon = self.EPSILON_INITIAL  # Exploration probability

        self.memory = ReplayBuffer(
            buffer_size=self.REPLAY_SIZE,
            observation_space=self.env.observation_space,
            action_space=self.env.action_space,
            device=device,
            optimize_memory_usage=True,
            handle_timeout_termination=False
        )

        self.timesteps = 0

        self.policy_network = DQN(4, self.env.action_space.n).to(device)
        self.target_network = DQN(4, self.env.action_space.n).to(device)
        self.target_network.load_state_dict(self.policy_network.state_dict())

        self.optimizer = torch.optim.Adam(self.policy_network.parameters(), lr=0.0001)
        self.loss_fn = nn.SmoothL1Loss()

        # Metrics/Logs
        self.PATH = "runs"
        if not os.path.exists(self.PATH):
            os.makedirs(self.PATH)

        self.CHECKPOINTS_PATH = f"{self.PATH}/checkpoints"
        self.LOGS_PATH = f"{self.PATH}/logs"
        self.VIDEO_PATH = f"{self.PATH}/videos"

        self.is_loaded_from_checkpoint = False
        self.writer = None

    def remember(self, observation, next_observation, action, reward, done, infos):
        self.memory.add(observation, next_observation, action, reward, done, infos)

    def act(self, state):
        # Reduce epsilon when learning started
        if self.timesteps >= self.LEARNING_STARTS:
            # Minus LEARNING_STARTS to takes into account that learning only started after LEARNING_STARTS,
            # and so we want to start reducing epsilon only when learning start
            self.epsilon = linear_schedule(
                self.EPSILON_INITIAL,
                self.EPSILON_FINAL,
                int(self.EXPLORATION_FRACTION * self.MAX_TIMESTEPS),
                self.timesteps - self.LEARNING_STARTS
            )

        if self.timesteps < self.LEARNING_STARTS or np.random.rand() < self.epsilon:
            # Random action
            return np.array(self.env.action_space.sample())
        else:
            with torch.no_grad():
                state_tensor = torch.tensor(np.array(state), device=device).unsqueeze(0)
                q_values = self.policy_network(state_tensor)
                return q_values.argmax(dim=1)[0].cpu().numpy()

    def update_target_network(self):
        self.target_network.load_state_dict(self.policy_network.state_dict())

    def optimize_model(self):
        minibatch = self.memory.sample(self.BATCH_SIZE)

        # Calculate Q values for current states
        # For each q_values, get the action according to the minibatch
        q_values = self.policy_network(minibatch.observations).gather(1, minibatch.actions)

        # Then, calculate the best actions for the next states, and return its indices
        with torch.no_grad():
            best_next_actions = self.policy_network(minibatch.next_observations).argmax(1).unsqueeze(1)

        # Calculate the Q values for the next states using the target network, and return the action according to the best next action returned by the q network
        target_next_q_values = self.target_network(minibatch.next_observations).gather(1, best_next_actions)

        # Calculate the target Q values using Double DQN
        target_q_values = minibatch.rewards + (1 - minibatch.dones) * self.GAMMA * target_next_q_values

        # Compute the loss
        loss = self.loss_fn(q_values, target_q_values)

        # Compute metrics for loss
        if self.timesteps % 100 == 0:
            self.writer.add_scalar("train/loss", loss, self.timesteps)
            self.writer.add_scalar("train/q_values", q_values.squeeze().mean().item(), self.timesteps)
            steps_per_second = int(self.timesteps / (time.time() - self.start_time))
            #print("Steps per second: ", steps_per_second)
            self.writer.add_scalar("train/steps_per_second", steps_per_second, self.timesteps)


        # Optimise Q network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def save_checkpoint(self):
        if self.start_datetime is None:
            print("SAVE_CHECKPOINT_ERROR: Training need to have started to save a checkpoint.")
            return

        print("Saving checkpoint...")
        current_datetime_str = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")
        start_datetime_str = self.start_datetime.strftime("%d-%m-%Y_%H:%M:%S")

        save_parent_directory = f"{self.CHECKPOINTS_PATH}/{self.name}_{start_datetime_str}"
        save_path = save_parent_directory + "/chkpt_" + current_datetime_str + ".tar"
        replay_buffer_path = save_parent_directory + "/replay_buffer_" + current_datetime_str

        if not os.path.exists(save_parent_directory):
            os.makedirs(save_parent_directory)

        checkpoint = {
            "env": self.env,
            "timesteps": self.timesteps,
            "start_datetime": self.start_datetime,
            "epsilon": self.epsilon,
            "policy_network": self.policy_network.state_dict(),
            "target_network": self.target_network.state_dict(),
            "optimizer": self.optimizer.state_dict(),
        }

        torch.save(checkpoint, save_path)
        # Saving the replay buffer will takes time! But it is needed to properly resume training
        save_to_pkl(replay_buffer_path, self.memory, verbose=1)

        print(f"Checkpoint saved into {save_parent_directory}")

    def load_last_checkpoint(self, path):
        """
        Load the last saved checkpoint found in the given ``path``

        :param path: the path to the directory containing the checkpoint(s)
        """
        print(f"Loading most recent checkpoint from {path}")
        self.is_loaded_from_checkpoint = True

        # Using list comprehension to filter directories and only get the files
        files = [file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))]

        checkpoint_files = [chkpt_file for chkpt_file in files if "chkpt" in chkpt_file]
        replay_buffer_files = [chkpt_file for chkpt_file in files if "replay_buffer" in chkpt_file]

        checkpoint_file = get_latest_checkpoint_file(checkpoint_files)
        replay_buffer_file = get_latest_checkpoint_file(replay_buffer_files)

        checkpoint: Dict[str, any] = torch.load(path + "/" + checkpoint_file)

        self.env = checkpoint["env"]
        self.timesteps = checkpoint["timesteps"]
        self.start_datetime: datetime = checkpoint["start_datetime"]
        self.start_time = self.start_datetime.timestamp()

        self.epsilon = checkpoint["epsilon"]

        self.policy_network.load_state_dict(checkpoint["policy_network"])
        self.target_network.load_state_dict(checkpoint["target_network"])
        self.optimizer.load_state_dict(checkpoint["optimizer"])

        self.memory: ReplayBuffer = load_from_pkl(path + "/" + replay_buffer_file)
        print("Checkpoint successfully loaded, you can resume the training now.")

    def run(self):
        # Either create a new SummaryWriter or resume from previous one
        if not self.is_loaded_from_checkpoint:
            current_datetime = datetime.now()
            self.start_datetime = current_datetime
            self.start_time = current_datetime.timestamp()

        start_datetime_str = self.start_datetime.strftime("%d-%m-%Y_%H:%M:%S")
        self.writer = SummaryWriter(f"{self.LOGS_PATH}/{self.name}_{start_datetime_str}")

        video_folder_path = f"{self.VIDEO_PATH}/{self.name}_{start_datetime_str}"
        if not os.path.exists(video_folder_path):
            os.makedirs(video_folder_path)
        self.env.video_folder = video_folder_path

        while self.timesteps < self.MAX_TIMESTEPS:
            state, _ = self.env.reset()
            done = False

            while not done:
                self.timesteps += 1

                action = self.act(state)
                next_state, reward, terminated, truncated, info = self.env.step(action)
                done = terminated or truncated

                self.remember(state, next_state, action, reward, terminated, info)

                if self.timesteps >= self.LEARNING_STARTS and self.timesteps % self.TRAIN_FREQUENCY == 0:
                    self.optimize_model()

                state = next_state

                if done:
                    mean_reward = np.mean(self.env.return_queue)
                    mean_length = np.mean(self.env.length_queue)

                    # Get episode statistics from info ("episode" key only exist when episode is done)
                    episode_reward = info["episode"]["r"]
                    self.writer.add_scalar("rollout/episodic_return", episode_reward, self.timesteps)
                    self.writer.add_scalar("rollout/episodic_length", info["episode"]["l"], self.timesteps)

                    self.writer.add_scalar("rollout/ep_rew_mean", mean_reward, self.timesteps)
                    self.writer.add_scalar("rollout/ep_len_mean", mean_length, self.timesteps)

                    self.writer.add_scalar("rollout/exploration_rate", self.epsilon, self.timesteps)


                    print(f"Episode {self.env.episode_count} finished (timesteps: {self.timesteps}/{self.MAX_TIMESTEPS})\n"
                          f"Epsilon: {self.epsilon:.2f}, Episode reward: {episode_reward.item()}, Mean reward: {mean_reward:.2f}")

                    if self.env.episode_count % self.CHECKPOINT_INTERVAL_EPISODE == 0:
                        self.save_checkpoint()
                    print("***************************")

                if self.timesteps >= self.LEARNING_STARTS and self.timesteps % self.TARGET_UPDATE_INTERVAL == 0:
                    self.update_target_network()
                    #print("Target model updated.")

        self.save_checkpoint() # Save last checkpoint at the end of training

        self.writer.flush()
        self.writer.close()

In [12]:
agent = DQNAgent()
#agent.load_last_checkpoint("runs/checkpoints/cnn_ddqn_ALE-Pong-v5_08-04-2023_20:00:00")
agent.run()

/home/deewens/miniconda3/envs/pytorch/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode 1 finished (timesteps: 974/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -20.00
***************************
Moviepy - Building video runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-0.mp4.
Moviepy - Writing video runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-0.mp4



Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-0.mp4
Episode 2 finished (timesteps: 1765/10000000)
Epsilon: 1.00, Episode reward: -21.0, Mean reward: -20.50
***************************
Episode 3 finished (timesteps: 2709/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -20.33
***************************
Episode 4 finished (timesteps: 3745/10000000)
Epsilon: 1.00, Episode reward: -19.0, Mean reward: -20.00
***************************
Episode 5 finished (timesteps: 4921/10000000)
Epsilon: 1.00, Episode reward: -18.0, Mean reward: -19.60
***************************
Episode 6 finished (timesteps: 5904/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -19.67
***************************
Episode 7 finished (timesteps: 6761/10000000)
Epsilon: 1.00, Episode reward: -20.0, Mean reward: -19.71
***************************
Episode 8 finished (timesteps: 7884/10000000)
Epsilon: 1.00, Episode reward: -19.0, Mean reward: -19

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-400000.mp4
Episode 418 finished (timesteps: 401688/10000000)
Epsilon: 0.70, Episode reward: -18.0, Mean reward: -19.70
***************************
Episode 419 finished (timesteps: 403103/10000000)
Epsilon: 0.70, Episode reward: -19.0, Mean reward: -19.69
***************************
Episode 420 finished (timesteps: 404483/10000000)
Epsilon: 0.70, Episode reward: -18.0, Mean reward: -19.67
***************************
Episode 421 finished (timesteps: 405514/10000000)
Epsilon: 0.70, Episode reward: -20.0, Mean reward: -19.66
***************************
Episode 422 finished (timesteps: 406508/10000000)
Epsilon: 0.70, Episode reward: -21.0, Mean reward: -19.67
***************************
Episode 423 finished (timesteps: 407710/10000000)
Epsilon: 0.70, Episode reward: -19.0, Mean reward: -19.66
***************************
Episode 424 finished (timesteps: 409105/10000000)
Epsilon: 0.69, Episod

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-800000.mp4
Episode 697 finished (timesteps: 801629/10000000)
Epsilon: 0.31, Episode reward: -16.0, Mean reward: -17.33
***************************
Episode 698 finished (timesteps: 803436/10000000)
Epsilon: 0.30, Episode reward: -14.0, Mean reward: -17.29
***************************
Episode 699 finished (timesteps: 805644/10000000)
Epsilon: 0.30, Episode reward: -12.0, Mean reward: -17.22
***************************
Episode 700 finished (timesteps: 807464/10000000)
Epsilon: 0.30, Episode reward: -17.0, Mean reward: -17.21
***************************
Episode 701 finished (timesteps: 809128/10000000)
Epsilon: 0.30, Episode reward: -17.0, Mean reward: -17.21
***************************
Episode 702 finished (timesteps: 810903/10000000)
Epsilon: 0.30, Episode reward: -13.0, Mean reward: -17.17
***************************
Episode 703 finished (timesteps: 812756/10000000)
Epsilon: 0.29, Episod

...

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-8400000.mp4
Episode 3591 finished (timesteps: 8401759/10000000)
Epsilon: 0.01, Episode reward: 20.0, Mean reward: 14.60
***************************
Episode 3592 finished (timesteps: 8404275/10000000)
Epsilon: 0.01, Episode reward: 16.0, Mean reward: 14.57
***************************
Episode 3593 finished (timesteps: 8406958/10000000)
Epsilon: 0.01, Episode reward: 10.0, Mean reward: 14.49
***************************
Episode 3594 finished (timesteps: 8409428/10000000)
Epsilon: 0.01, Episode reward: 11.0, Mean reward: 14.48
***************************
Episode 3595 finished (timesteps: 8411433/10000000)
Epsilon: 0.01, Episode reward: 18.0, Mean reward: 14.59
***************************
Episode 3596 finished (timesteps: 8413123/10000000)
Epsilon: 0.01, Episode reward: 20.0, Mean reward: 14.72
***************************
Episode 3597 finished (timesteps: 8415212/10000000)
Epsilon: 0.01, Epi

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-8800000.mp4
Episode 3778 finished (timesteps: 8801418/10000000)
Epsilon: 0.01, Episode reward: 17.0, Mean reward: 16.38
***************************
Episode 3779 finished (timesteps: 8803382/10000000)
Epsilon: 0.01, Episode reward: 18.0, Mean reward: 16.45
***************************
Episode 3780 finished (timesteps: 8805322/10000000)
Epsilon: 0.01, Episode reward: 18.0, Mean reward: 16.51
***************************
Episode 3781 finished (timesteps: 8807910/10000000)
Epsilon: 0.01, Episode reward: 14.0, Mean reward: 16.50
***************************
Episode 3782 finished (timesteps: 8810640/10000000)
Epsilon: 0.01, Episode reward: 13.0, Mean reward: 16.45
***************************
Episode 3783 finished (timesteps: 8812779/10000000)
Epsilon: 0.01, Episode reward: 16.0, Mean reward: 16.46
***************************
Episode 3784 finished (timesteps: 8814563/10000000)
Epsilon: 0.01, Epi

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-9200000.mp4
Episode 3969 finished (timesteps: 9201416/10000000)
Epsilon: 0.01, Episode reward: 19.0, Mean reward: 16.39
***************************
Episode 3970 finished (timesteps: 9203577/10000000)
Epsilon: 0.01, Episode reward: 13.0, Mean reward: 16.34
***************************
Episode 3971 finished (timesteps: 9205336/10000000)
Epsilon: 0.01, Episode reward: 21.0, Mean reward: 16.40
***************************
Episode 3972 finished (timesteps: 9207419/10000000)
Epsilon: 0.01, Episode reward: 16.0, Mean reward: 16.44
***************************
Episode 3973 finished (timesteps: 9209112/10000000)
Epsilon: 0.01, Episode reward: 20.0, Mean reward: 16.51
***************************
Episode 3974 finished (timesteps: 9210913/10000000)
Epsilon: 0.01, Episode reward: 20.0, Mean reward: 16.52
***************************
Episode 3975 finished (timesteps: 9213134/10000000)
Epsilon: 0.01, Epi

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-9600000.mp4
Episode 4157 finished (timesteps: 9602703/10000000)
Epsilon: 0.01, Episode reward: 20.0, Mean reward: 16.66
***************************
Episode 4158 finished (timesteps: 9605062/10000000)
Epsilon: 0.01, Episode reward: 15.0, Mean reward: 16.62
***************************
Episode 4159 finished (timesteps: 9606768/10000000)
Epsilon: 0.01, Episode reward: 20.0, Mean reward: 16.66
***************************
Episode 4160 finished (timesteps: 9609005/10000000)
Epsilon: 0.01, Episode reward: 17.0, Mean reward: 16.65
***************************
Episode 4161 finished (timesteps: 9611364/10000000)
Epsilon: 0.01, Episode reward: 18.0, Mean reward: 16.65
***************************
Episode 4162 finished (timesteps: 9613570/10000000)
Epsilon: 0.01, Episode reward: 18.0, Mean reward: 16.64
***************************
Episode 4163 finished (timesteps: 9615976/10000000)
Epsilon: 0.01, Epi

Moviepy - Done !
Moviepy - video ready runs/videos/CNN_DDQN_Pong-v5_11-04-2023_19:37:01/rl-video-step-10000000.mp4
Episode 4348 finished (timesteps: 10001160/10000000)
Epsilon: 0.01, Episode reward: 16.0, Mean reward: 16.33
***************************
Saving checkpoint...
Checkpoint saved into runs/checkpoints/CNN_DDQN_Pong-v5_11-04-2023_19:37:01
